# Assignment 2: XAI
Task: Predict the diagnosis for given x-ray images
- Task Information: https://www.kaggle.com/datasets/paultimothymooney/chest-xray-pneumonia/
- Data Information: https://www.kaggle.com/datasets/paultimothymooney/chest-xray-pneumonia/data
Report
- Task definition
- Data description (results of data exploration)
- Description of baseline method
- Desciption of hyperparameter optimization
- Results: A table for...
    - Baseline results (repication of exisiting method)
    - Results of hyperparameter optimization/alternative approach
    - Extensive analysis of best model using XAI techniques
- Brief conclusion: Which feature respresentation/model worked the best and why

## Code

Imports

In [ ]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("paultimothymooney/chest-xray-pneumonia")

print("Path to dataset files:", path)

In [ ]:
import numpy as np 
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns
import keras
from keras.models import Sequential
from keras.layers import Dense, Conv2D , MaxPool2D , Flatten , Dropout , BatchNormalization
from keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report,confusion_matrix
from keras.callbacks import ReduceLROnPlateau
import cv2
import os

for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))


Resuming download from 417333248 bytes (2046032187 bytes left)...
Resuming download from https://www.kaggle.com/api/v1/datasets/download/paultimothymooney/chest-xray-pneumonia?dataset_version_number=2 (417333248/2463365435) bytes left.


 25%|██▍       | 581M/2.29G [00:17<02:49, 10.9MB/s]


KeyboardInterrupt: 

### 1.0 Dataset Exploration
- Download dataset from Kaggle
- Explore dataset to under input features (images), classes (normal vs. pneumonia), and distribution of data
- Visualize sample images for both clases, along with their labels
    - Can use https://www.kaggle.com/code/madz2000/pneumonia-detection-using-cnn-92-6-accuracy as reference (do not just copy)

### 2.0 CNN Baseline
- Use example CNN architecture provide in Kaggle code as a base. Understand role of key components like convolutional layers, pooling layers, and fully connected layers
- Train model on training dataset
- Track and report model performance metrics (accuracy, loss) during training and on the test dataset

### 3.0 Performance Evaluation
- After training, evaluate model on test set
- Present the confusion matrix, precision, recall, and F1 score for baseline model


### 4.0 Hyperparameter Optimization & Alternative Models
- Use an HPO method (SMAC3, Bayesian optimization, etc) to try to improve the current model; use a realistic budget (do no run experiments for more than a day)
- Try a different model (machline learning technique) for the given task, provide motivation as to why
- Present confusion matrix, precision, recall, and F1 score on train and test set for the final optimized CNN model and the alternative ML model

### 4.0 Explainable AI
- Inspect misclassification errors and use explainable AI techniques to highlight what the models are focusing on
- Feature importance with Grad-CAM (example at https://keras.io/examples/vision/grad_cam/, https://github.com/ismailuddin/gradcam-tensorflow-2, or https://deel-ai.github.io/xplique/v1.4.0/)
    - Use Grad-CAM (Gradient-weighted Class Activation Mapping) to visualize which parts of the input images contribute most to the model's predictions
    - Choose 5-10 misclassified samples and use Grad-CAM to inspect which regions the model is focusing on
    - Interpret the results:
        - Are the activations reasonable?
        - Can it be identified why the model made incorrect predictions for chosen samples?
    - Layer-wise Relevance Propagation (LRP) (optional for bonus points)
        - Apply LRP to further analyze how different layers of network contribute to the final predictions
        - Compare insights from LRP with Grad-CAM, are there any differences in how each method highlights the important regions of the image?

## Report
- Task definition
- Data description (results of data exploration)
- Description of baseline method
- Desciption of hyperparameter optimization
- Results: A table for...
    - Baseline results (repication of exisiting method)
    - Results of hyperparameter optimization/alternative approach
    - Extensive analysis of best model using XAI techniques
- Brief conclusion: Which feature respresentation/model worked the best and why
<br>
Maximum 8 pages

## Submission
Submit report as pdf along with code zipfile in Brightspace